In [1]:
# achieve property prediction using machine learning,
# When achieve predicting task for a property, only select data with that property measured (Non-NaN) 


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error


In [4]:
# Load dataset
data = pd.read_csv('datasets/Formula/processed_formula_property_data.csv') 
data

,FID,Property_1,Property_2,Property_3,Property_4,Property_5,Property_6,Property_7,Property_8,Property_9,...,Formula_PC29,Formula_PC30,Formula_PC31,Formula_PC32,Formula_PC33,Formula_PC34,Formula_PC35,Condition_PC1,Condition_PC2,Condition_PC3
0,1,NaN,NaN,0.187732,NaN,NaN,NaN,0.192958,NaN,NaN,...,0.630510,-1.044794,-1.010974,0.784127,-0.785801,0.434642,-1.168980,-0.552511,-0.215972,0.326480
1,2,NaN,NaN,0.167026,NaN,NaN,NaN,0.178770,NaN,NaN,...,0.626488,-1.039497,-1.011796,0.781721,-0.779595,0.437224,-1.161499,-0.552511,-0.215972,0.326480
2,3,NaN,NaN,0.207747,NaN,NaN,NaN,0.172320,NaN,NaN,...,0.625965,-1.040429,-1.009537,0.782721,-0.782244,0.433595,-1.167042,-0.552511,-0.215972,0.326480
3,4,NaN,NaN,0.318178,NaN,NaN,NaN,0.430285,NaN,NaN,...,0.944698,-0.966308,-1.208546,1.190928,-0.231057,1.037499,-1.816077,-0.552511,-0.215972,0.326480
4,5,NaN,NaN,0.314727,NaN,NaN,NaN,0.418935,NaN,NaN,...,0.962751,-0.445088,-1.135749,0.909521,0.174548,1.176918,-1.131358,-0.552511,-0.215972,0.326480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9905,5507,0.253754,0.300485,0.247778,0.268624,0.226766,0.368019,0.552818,0.244909,0.681583,...,-0.013889,0.703369,-0.138177,0.232650,0.012936,-0.055710,-0.386364,0.126704,0.526293,-0.720117
9906,5508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.098897,1.631745,-0.126407,0.355267,0.879366,0.325944,-0.645177,-0.552511,-0.215972,0.326480
9907,5509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.328240,-1.108290,0.176624,-1.003014,0.223545,-0.215317,1.043960,-0.552511,-0.215972,0.326480
9908,5510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.233434,-0.102092,0.146241,-1.154091,0.553121,0.201320,1.213065,-0.552511,-0.215972,0.326480


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# Load dataset
data = pd.read_csv('datasets/Formula/processed_formula_property_data.csv')

# Display basic info about the dataset
print(f"Dataset shape: {data.shape}")
print(f"Expected columns: FID + 66 Properties + 35 Formula_PCs + 3 Condition_PCs = 105 columns")
print(f"Actual columns: {len(data.columns)}")

# Check for the expected structure
expected_properties = [f'Property_{i}' for i in range(1, 67)]
expected_formula_features = [f'Formula_PC{i}' for i in range(1, 36)]
expected_condition_features = [f'Condition_PC{i}' for i in range(1, 4)]

properties_present = [col for col in expected_properties if col in data.columns]
formula_features_present = [col for col in expected_formula_features if col in data.columns]
condition_features_present = [col for col in expected_condition_features if col in data.columns]

print(f"\nFound {len(properties_present)} properties out of 66 expected")
print(f"Found {len(formula_features_present)} formula features out of 35 expected")
print(f"Found {len(condition_features_present)} condition features out of 3 expected")

if 'FID' in data.columns:
    print("FID column found")
else:
    print("Warning: FID column not found")

print(f"\nSample of data:")
print(data.head())

# Analyze the dataset structure
print("Dataset Info:")
print(data.info())
print(f"\nMissing values per column:")
print(data.isnull().sum())


class PropertyPredictor:
    def __init__(self, data):
        self.data = data.copy()
        self.feature_columns = None
        self.property_columns = None
        self.models = {
            'Random Forest': RandomForestRegressor(random_state=42),
            'Gradient Boosting': GradientBoostingRegressor(random_state=42),
            'Support Vector Regression': SVR(),
            'Linear Regression': LinearRegression(),
            'Ridge Regression': Ridge(random_state=42),
            'Decision Tree': DecisionTreeRegressor(random_state=42),
            'K-Nearest Neighbors': KNeighborsRegressor()
        }
        self.results = {}

    def identify_columns(self, feature_prefixes=None, property_prefixes=None):
        """
        Identify feature and property columns based on prefixes
        """
        if feature_prefixes is None or property_prefixes is None:
            print("Error: Please specify feature_prefixes and property_prefixes")
            return

        # Find columns matching the prefixes
        feature_cols = [col for col in self.data.columns
                        if any(col.startswith(prefix) for prefix in feature_prefixes)]
        property_cols = [col for col in self.data.columns
                         if any(col.startswith(prefix) for prefix in property_prefixes)]

        # Sort the columns for better organization
        feature_cols = sorted(feature_cols)
        property_cols = sorted(property_cols, key=lambda x: int(x.split('_')[1]) if '_' in x else 0)

        self.feature_columns = feature_cols
        self.property_columns = property_cols

        print(f"Found {len(feature_cols)} feature columns:")
        print(f"  Formula features: {len([c for c in feature_cols if 'Formula' in c])}")
        print(f"  Condition features: {len([c for c in feature_cols if 'Condition' in c])}")
        print(f"Found {len(property_cols)} property columns (Property_1 to Property_{len(property_cols)})")

        # Show sample of columns
        print(f"\nSample feature columns: {feature_cols[:5]}...")
        print(f"Sample property columns: {property_cols[:5]}...")

    def prepare_data_for_property(self, target_property):
        """
        Prepare dataset for a specific property prediction (only non-NaN values)
        """
        # Filter data where target property is not NaN
        valid_data = self.data.dropna(subset=[target_property])

        # Get features (drop rows where any feature is NaN)
        X = valid_data[self.feature_columns].dropna()
        y = valid_data.loc[X.index, target_property]

        print(f"\nProperty: {target_property}")
        print(f"Available samples: {len(X)} out of {len(self.data)}")
        print(f"Feature shape: {X.shape}")

        return X, y

    def evaluate_models(self, X, y, property_name):
        """
        Evaluate multiple models for a given property
        """
        results = {}

        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        print(f"\nEvaluating models for {property_name}...")
        print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")

        for name, model in self.models.items():
            try:
                # Create pipeline with scaling for models that need it
                if name in ['Support Vector Regression', 'K-Nearest Neighbors']:
                    pipeline = make_pipeline(StandardScaler(), model)
                else:
                    pipeline = model

                # Train the model
                pipeline.fit(X_train, y_train)

                # Make predictions
                y_pred = pipeline.predict(X_test)

                # Calculate metrics
                mse = mean_squared_error(y_test, y_pred)
                rmse = np.sqrt(mse)
                mae = mean_absolute_error(y_test, y_pred)
                r2 = r2_score(y_test, y_pred)

                # Cross-validation score
                cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5,
                                            scoring='r2', n_jobs=-1)

                results[name] = {
                    'MSE': mse,
                    'RMSE': rmse,
                    'MAE': mae,
                    'R2': r2,
                    'CV_R2_mean': cv_scores.mean(),
                    'CV_R2_std': cv_scores.std(),
                    'model': pipeline,
                    'predictions': y_pred,
                    'y_test': y_test
                }

                print(
                    f"{name:25} - R2: {r2:.4f}, RMSE: {rmse:.4f}, CV R2: {cv_scores.mean():.4f}±{cv_scores.std():.4f}")

            except Exception as e:
                print(f"Error with {name}: {str(e)}")
                continue

        return results

    def plot_results(self, results, property_name):
        """
        Plot model comparison and predictions
        """
        if not results:
            print("No results to plot")
            return

        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle(f'Model Evaluation for {property_name}', fontsize=16)

        # 1. Model comparison barplot
        model_names = list(results.keys())
        r2_scores = [results[name]['R2'] for name in model_names]
        rmse_scores = [results[name]['RMSE'] for name in model_names]

        axes[0, 0].barh(model_names, r2_scores)
        axes[0, 0].set_xlabel('R² Score')
        axes[0, 0].set_title('Model R² Comparison')
        axes[0, 0].set_xlim(0, 1)

        # 2. RMSE comparison
        axes[0, 1].barh(model_names, rmse_scores)
        axes[0, 1].set_xlabel('RMSE')
        axes[0, 1].set_title('Model RMSE Comparison')

        # 3. Best model predictions vs actual
        best_model_name = max(results.keys(), key=lambda x: results[x]['R2'])
        best_result = results[best_model_name]

        axes[1, 0].scatter(best_result['y_test'], best_result['predictions'], alpha=0.6)
        axes[1, 0].plot([best_result['y_test'].min(), best_result['y_test'].max()],
                        [best_result['y_test'].min(), best_result['y_test'].max()], 'r--')
        axes[1, 0].set_xlabel('Actual Values')
        axes[1, 0].set_ylabel('Predicted Values')
        axes[1, 0].set_title(f'Best Model: {best_model_name}\nR² = {best_result["R2"]:.4f}')

        # 4. Residuals plot
        residuals = best_result['y_test'] - best_result['predictions']
        axes[1, 1].scatter(best_result['predictions'], residuals, alpha=0.6)
        axes[1, 1].axhline(y=0, color='r', linestyle='--')
        axes[1, 1].set_xlabel('Predicted Values')
        axes[1, 1].set_ylabel('Residuals')
        axes[1, 1].set_title('Residuals Plot')

        plt.tight_layout()
        plt.show()

        return best_model_name, best_result

    def hyperparameter_tuning(self, X, y, property_name, model_name='Random Forest'):
        """
        Perform hyperparameter tuning for the best performing model
        """
        print(f"\nPerforming hyperparameter tuning for {model_name}...")

        if model_name == 'Random Forest':
            model = RandomForestRegressor(random_state=42)
            param_grid = {
                'n_estimators': [50, 100, 200],
                'max_depth': [None, 10, 20],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4]
            }
        elif model_name == 'Gradient Boosting':
            model = GradientBoostingRegressor(random_state=42)
            param_grid = {
                'n_estimators': [50, 100, 200],
                'learning_rate': [0.01, 0.1, 0.2],
                'max_depth': [3, 5, 7]
            }
        else:
            print(f"Hyperparameter tuning not implemented for {model_name}")
            return None

        # Grid search
        grid_search = GridSearchCV(
            model, param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=1
        )

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        grid_search.fit(X_train, y_train)

        # Best model evaluation
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)

        print(f"Best parameters: {grid_search.best_params_}")
        print(f"Best CV score: {grid_search.best_score_:.4f}")
        print(f"Test R²: {r2_score(y_test, y_pred):.4f}")

        return best_model, grid_search.best_params_

    def feature_importance_analysis(self, model, feature_names, property_name):
        """
        Analyze and plot feature importance
        """
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            indices = np.argsort(importances)[::-1]

            plt.figure(figsize=(12, 8))
            plt.title(f'Feature Importance for {property_name}')
            plt.barh(range(len(importances)), importances[indices])
            plt.yticks(range(len(importances)), [feature_names[i] for i in indices])
            plt.xlabel('Importance')
            plt.gca().invert_yaxis()
            plt.tight_layout()
            plt.show()

            # Print top 10 features
            print(f"\nTop 10 most important features for {property_name}:")
            for i in range(min(10, len(indices))):
                idx = indices[i]
                print(f"{i + 1:2d}. {feature_names[idx]:30s} {importances[idx]:.4f}")
        else:
            print("Model doesn't have feature importance attribute")

    def predict_all_properties(self):
        """
        Run prediction pipeline for all properties
        """
        if self.property_columns is None:
            print("Please run identify_columns() first")
            return

        all_results = {}

        for property_name in self.property_columns:
            print(f"\n{'=' * 60}")
            print(f"PROCESSING PROPERTY: {property_name}")
            print(f"{'=' * 60}")

            try:
                # Prepare data
                X, y = self.prepare_data_for_property(property_name)

                if len(X) < 10:  # Skip if too few samples
                    print(f"Skipping {property_name} - insufficient samples ({len(X)})")
                    continue

                # Evaluate models
                results = self.evaluate_models(X, y, property_name)

                # Plot results
                best_model_name, best_result = self.plot_results(results, property_name)

                # Feature importance for tree-based models
                if hasattr(best_result['model'], 'feature_importances_'):
                    self.feature_importance_analysis(
                        best_result['model'], self.feature_columns, property_name
                    )

                # Store results
                all_results[property_name] = {
                    'results': results,
                    'best_model': best_model_name,
                    'best_score': best_result['R2']
                }

            except Exception as e:
                print(f"Error processing {property_name}: {str(e)}")
                continue

        # Summary
        print(f"\n{'=' * 60}")
        print("SUMMARY")
        print(f"{'=' * 60}")

        for prop, info in all_results.items():
            print(f"{prop:30s} - Best: {info['best_model']:20s} (R² = {info['best_score']:.4f})")

    def predict_specific_properties(self, property_list):
        """
        Run prediction pipeline for specific properties only
        """
        if self.property_columns is None:
            print("Please run identify_columns() first")
            return

        # Filter to only requested properties that exist
        valid_properties = [prop for prop in property_list if prop in self.property_columns]
        invalid_properties = [prop for prop in property_list if prop not in self.property_columns]

        if invalid_properties:
            print(f"Warning: These properties not found: {invalid_properties}")

        if not valid_properties:
            print("No valid properties found")
            return

        print(f"Processing {len(valid_properties)} properties: {valid_properties}")

        results = {}

        for property_name in valid_properties:
            print(f"\n{'=' * 60}")
            print(f"PROCESSING PROPERTY: {property_name}")
            print(f"{'=' * 60}")

            try:
                # Prepare data
                X, y = self.prepare_data_for_property(property_name)

                if len(X) < 10:  # Skip if too few samples
                    print(f"Skipping {property_name} - insufficient samples ({len(X)})")
                    continue

                # Evaluate models
                property_results = self.evaluate_models(X, y, property_name)

                # Plot results
                best_model_name, best_result = self.plot_results(property_results, property_name)

                # Feature importance for tree-based models
                if hasattr(best_result['model'], 'feature_importances_'):
                    self.feature_importance_analysis(
                        best_result['model'], self.feature_columns, property_name
                    )

                # Store results
                results[property_name] = {
                    'results': property_results,
                    'best_model': best_model_name,
                    'best_score': best_result['R2']
                }

            except Exception as e:
                print(f"Error processing {property_name}: {str(e)}")
                continue

        # Summary
        print(f"\n{'=' * 60}")
        print("SUMMARY")
        print(f"{'=' * 60}")

        for prop, info in results.items():
            print(f"{prop:15s} - Best: {info['best_model']:20s} (R² = {info['best_score']:.4f})")

        return results


# Initialize the predictor
predictor = PropertyPredictor(data)

# Identify feature and property columns
# Identify columns based on your specific structure
feature_prefixes = ['Formula_PC', 'Condition_PC']  # Features are Formula_PC* and Condition_PC*
property_prefixes = ['Property_']  # Properties are Property_*
predictor.identify_columns(feature_prefixes, property_prefixes)

# Run prediction for all properties
results = predictor.predict_all_properties()

# Optional: Detailed analysis for a specific property (e.g., Property_1)
specific_property = 'Property_1'  # You can change this to any Property_X
print(f"\nDetailed analysis for: {specific_property}")

if specific_property in predictor.property_columns:
    X, y = predictor.prepare_data_for_property(specific_property)
    if len(X) > 10:  # Only proceed if enough samples
        results_detailed = predictor.evaluate_models(X, y, specific_property)

        # Hyperparameter tuning for best model
        best_model_name = max(results_detailed.keys(), key=lambda x: results_detailed[x]['R2'])
        best_model, best_params = predictor.hyperparameter_tuning(X, y, specific_property, best_model_name)
    else:
        print(f"Insufficient samples for {specific_property}")
else:
    print(f"{specific_property} not found in property columns")


# Data exploration and correlation analysis
def explore_data(data, property_columns, feature_columns):
    """
    Explore the dataset and show correlations
    """
    print("Data Exploration:")
    print(f"Total samples: {len(data)}")
    print(f"Total features: {len(feature_columns)}")
    print(f"Total properties: {len(property_columns)}")

    # Correlation matrix for properties
    if len(property_columns) > 1:
        plt.figure(figsize=(10, 8))
        correlation_matrix = data[property_columns].corr()
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
        plt.title('Property Correlation Matrix')
        plt.tight_layout()
        plt.show()

    # Distribution of each property
    fig, axes = plt.subplots((len(property_columns) + 2) // 3, 3,
                             figsize=(15, 5 * ((len(property_columns) + 2) // 3)))
    axes = axes.flatten() if len(property_columns) > 1 else [axes]

    for i, prop in enumerate(property_columns):
        if i < len(axes):
            data[prop].dropna().hist(bins=30, ax=axes[i])
            axes[i].set_title(f'Distribution of {prop}')
            axes[i].set_xlabel(prop)
            axes[i].set_ylabel('Frequency')

    # Hide unused subplots
    for i in range(len(property_columns), len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()


# Run data exploration
explore_data(data, predictor.property_columns, predictor.feature_columns)

# Check property data availability
property_stats = predictor.get_property_statistics()

# Example: Predict only first 10 properties
first_10_properties = [f'Property_{i}' for i in range(1, 11)]
results_subset = predictor.predict_specific_properties(first_10_properties)

# Example: Focus on specific properties of interest
# properties_of_interest = ['Property_1', 'Property_5', 'Property_10', 'Property_15']
# results_specific = predictor.predict_specific_properties(properties_of_interest)

print("\nPipeline completed! Check the results and plots above.")
print("\nQuick tips:")
print("- Use predict_all_properties() to process all 66 properties (may take time)")
print("- Use predict_specific_properties([list]) to focus on specific properties")
print("- Use get_property_statistics() to see data availability")
print("- Check the feature importance plots to understand what drives each property")

Dataset shape: (9910, 105)
Expected columns: FID + 66 Properties + 35 Formula_PCs + 3 Condition_PCs = 105 columns
Actual columns: 105

Found 66 properties out of 66 expected
Found 35 formula features out of 35 expected
Found 3 condition features out of 3 expected
FID column found

Sample of data:
   FID  Property_1  Property_2  Property_3  Property_4  Property_5  \
0    1         NaN         NaN    0.187732         NaN         NaN   
1    2         NaN         NaN    0.167026         NaN         NaN   
2    3         NaN         NaN    0.207747         NaN         NaN   
3    4         NaN         NaN    0.318178         NaN         NaN   
4    5         NaN         NaN    0.314727         NaN         NaN   

   Property_6  Property_7  Property_8  Property_9  ...  Formula_PC29  \
0         NaN    0.192958         NaN         NaN  ...      0.630510   
1         NaN    0.178770         NaN         NaN  ...      0.626488   
2         NaN    0.172320         NaN         NaN  ...      0.625


KeyboardInterrupt

